# Convierte publicación de temperaturas del SMN en series de tiempo

In [62]:
import pandas as pd
import arrow
import os
import requests
import re

In [64]:
os.makedirs("datos/output", exist_ok=True)
os.makedirs("datos/input", exist_ok=True)

# inputs
URL_ESTACIONES = "https://ssl.smn.gob.ar/dpd/zipopendata.php?dato=estaciones"
URL_TEMPERATURAS = "https://ssl.smn.gob.ar/dpd/zipopendata.php?dato=regtemp"
PATH_EST_BACKUP = "datos/input/estaciones-backup.csv"
PATH_TEMP_BACKUP = "datos/input/temperaturas-backup.csv"

# outputs
PATH_TEMP_MAX = "datos/output/temperaturas-maximas.csv"
PATH_TEMP_MIN = "datos/output/temperaturas-minimas.csv"
PATH_TEMP_PANEL = "datos/output/temperaturas-panel.csv"
PATH_ESTACIONES = "datos/output/estacioes-meteorologicas.csv"

## Lee y normaliza estaciones

In [71]:
def get_estaciones(url_estaciones=URL_ESTACIONES, 
                   path_est_backup=PATH_EST_BACKUP):
    """Lee y mantiene un backup de estaciones meteorológicas."""
    
    estaciones = pd.read_fwf(
        url_estaciones, compression="zip", encoding="latin1", skiprows=[1], 
        dtype={"FECHA": str}
    )
    try:
        estaciones_backup = pd.read_csv(path_est_backup, encoding="utf8")
        estaciones_total = pd.concat([estaciones, estaciones_backup]).drop_duplicates()
        estaciones_total.to_csv(path_est_backup, encoding="utf8", index=False)
    except:
        estaciones_total = estaciones
    return estaciones_total

In [72]:
estaciones = get_estaciones()
estaciones.head()

,NOMBRE,PROVINCIA,LATITUD,LONGITUD,ALTURA,NRO,NroOACI
0,BASE BELGRANO II,ANTARTIDA,-77 52,-34 34,256.0,89034.0,SAYB
1,BASE CARLINI (EX JUBANY),ANTARTIDA,-62 14,-58 38,11.0,89053.0,SAYJ
2,BASE ESPERANZA,ANTARTIDA,-63 24,-56 59,24.0,88963.0,SAYE
3,BASE MARAMBIO,ANTARTIDA,-64 14,-56 43,198.0,89055.0,SAWB
4,BASE ORCADAS,ANTARTIDA,-60 45,-44 43,12.0,88968.0,SAYO


In [77]:
# genera diccionario para convertir ids de estaciones en nombres
estaciones_dict = {
    row[1]["NroOACI"]: row[1]["NOMBRE"] 
    for row in estaciones[["NOMBRE", "NroOACI"]].iterrows()
}

In [78]:
def get_unidades_territoriales(latitud, longitud):
    """Busca unidades territoriales a partir de las coordenadas."""
    try:
        latitud = ".".join(re.split("\s+", latitud, maxsplit=2))
        longitud = ".".join(re.split("\s+", longitud, maxsplit=2))

        r = requests.get("https://apis.datos.gob.ar/georef/api/ubicacion?lat={lat}&lon={lon}".format(lat=latitud, lon=longitud))
        return r.json()["ubicacion"]
    
    except Exception as e:
        print(e)
        print("No se pudo localizar lat:{} y lon:{}".format(latitud, longitud))
        return None

In [80]:
# extrae ids y nombres oficiales de provincias y departamentos, para cada estación
estaciones["api_georef_ubicacion"] = estaciones.apply(
    lambda row: get_unidades_territoriales(row["LATITUD"], row["LONGITUD"]), 
    axis=1
)

estaciones["provincia_id"] = estaciones.api_georef_ubicacion.apply(lambda x: x["provincia"]["id"] if x else None)
estaciones["provincia_nombre"] = estaciones.api_georef_ubicacion.apply(lambda x: x["provincia"]["nombre"] if x else None)
estaciones["departamento_id"] = estaciones.api_georef_ubicacion.apply(lambda x: x["departamento"]["id"] if x else None)
estaciones["departamento_nombre"] = estaciones.api_georef_ubicacion.apply(lambda x: x["departamento"]["nombre"] if x else None)

expected string or bytes-like object
No se pudo localizar lat:nan y lon:nan
expected string or bytes-like object
No se pudo localizar lat:nan y lon:nan
expected string or bytes-like object
No se pudo localizar lat:nan y lon:nan
expected string or bytes-like object
No se pudo localizar lat:nan y lon:nan


In [81]:
estaciones_normalizado = estaciones.drop(columns=[
    "api_georef_ubicacion", "PROVINCIA"
]).rename(columns={
    "NOMBRE": "estacion_nombre",
    "LATITUD": "estacion_latitud",
    "LONGITUD": "estacion_longitud",
    "ALTURA": "estacion_altura",
    "NRO": "estacion_id",
    "NroOACI": "estacion_oaci_id"
})

estaciones_normalizado.to_csv(
    PATH_ESTACIONES, encoding="utf8", index=False, float_format='%.0f')

## Lee y normaliza temperaturas

In [82]:
def get_temperaturas(url_temperaturas=URL_TEMPERATURAS, 
                     path_temp_backup=PATH_TEMP_BACKUP):
    """Lee y mantiene un backup incremental de temperaturas diarias."""
    
    temperaturas = pd.read_fwf(
        url_temperaturas, compression="zip", encoding="latin1", skiprows=[2], 
        header=1, dtype={"FECHA": str})
    try:
        temperaturas_backup = pd.read_csv(path_temp_backup, encoding="utf8")
        temperaturas_total = pd.concat([temperaturas, temperaturas_backup]).drop_duplicates()
        temperaturas_total.to_csv(path_temp_backup, encoding="utf8", index=False)
    except:
        temperaturas_total = temperaturas
    return temperaturas_total

In [83]:
temperaturas = get_temperaturas()
temperaturas.head()

,FECHA,TMAX,TMIN,NOMBRE
0,16112019,25.2,18.2,AEROPARQUE AERO
1,16112019,31.5,16.5,AZUL AERO
2,16112019,29.3,13.8,BAHIA BLANCA AERO
3,16112019,15.4,6.8,BARILOCHE AERO
4,16112019,-3.0,-10.0,BASE BELGRANO II


In [84]:
def get_temperaturas_estaciones(temperaturas, estaciones):
    temperaturas_estaciones = temperaturas.merge(estaciones[["NOMBRE", "NroOACI"]], on="NOMBRE")[["FECHA", "TMAX", "TMIN", "NroOACI"]]
    temperaturas_estaciones["FECHA"] = temperaturas_estaciones["FECHA"].apply(lambda x: arrow.get(x, "DDMMYYYY").format("YYYY-MM-DD"))
    return temperaturas_estaciones

In [85]:
temperaturas_estaciones = get_temperaturas_estaciones(temperaturas, estaciones)
temperaturas_estaciones.head()

,FECHA,TMAX,TMIN,NroOACI
0,2019-11-16,25.2,18.2,SABE
1,2019-11-15,26.0,19.8,SABE
2,2019-11-14,30.2,20.3,SABE
3,2019-11-13,27.0,20.4,SABE
4,2019-11-12,25.0,18.8,SABE


In [87]:
# crea panel normalizado de temperaturas
temperaturas_normalizado = temperaturas_estaciones.rename(columns={
    "FECHA": "fecha",
    "TMAX": "temperatura_maxima",
    "TMIN": "temperatura_minima",
    "NroOACI": "estacion_oaci_id"
})

temperaturas_normalizado.to_csv(PATH_TEMP_PANEL, encoding="utf8", index=False)

## Convierte paneles en series

In [88]:
def rename_columns(col, prefix):
    if col == "FECHA":
        return "indice_tiempo"
    else:
        return "{}_{}".format(prefix.lower(), col.lower())

In [89]:
def temperatures_panel_to_series(df_panel, field_values="TMAX", prefix="temperatura_maxima"):
    df_series = df_panel.pivot_table(index="FECHA", values=field_values, columns="NroOACI").reset_index().sort_values("FECHA")
    df_series.columns = [rename_columns(col, prefix) for col in df_series.columns]
    return df_series

In [90]:
temp_max_series = temperatures_panel_to_series(temperaturas_estaciones, "TMAX", "temperatura_maxima")
temp_min_series = temperatures_panel_to_series(temperaturas_estaciones, "TMIN", "temperatura_minima")

In [91]:
temp_min_series.head()

,indice_tiempo,temperatura_minima_saac,temperatura_minima_saag,temperatura_minima_saai,temperatura_minima_saaj,temperatura_minima_saap,temperatura_minima_saar,temperatura_minima_saav,temperatura_minima_saba,temperatura_minima_sabe,...,temperatura_minima_sazm,temperatura_minima_sazn,temperatura_minima_sazp,temperatura_minima_sazq,temperatura_minima_sazr,temperatura_minima_sazs,temperatura_minima_sazt,temperatura_minima_sazv,temperatura_minima_sazx,temperatura_minima_sazy
0,2018-11-17,18.8,16.2,15.0,15.0,18.3,20.0,20.0,17.8,18.3,...,11.8,12.0,13.0,10.8,10.5,4.4,10.0,10.0,13.9,4.2
1,2018-11-18,16.0,13.2,11.5,11.8,13.6,12.4,13.5,13.2,15.0,...,9.9,10.2,9.4,7.6,6.1,5.5,7.3,9.0,9.7,4.2
2,2018-11-19,9.5,11.8,13.0,9.2,11.4,10.1,13.1,15.8,16.0,...,9.3,8.2,11.4,10.4,11.6,7.1,7.5,11.0,12.2,7.3
3,2018-11-20,10.8,13.4,16.0,13.1,13.5,13.6,14.9,18.2,18.1,...,15.6,11.4,16.5,13.6,15.9,9.4,14.0,16.5,15.8,10.2
4,2018-11-21,15.5,15.5,17.4,16.8,17.8,16.7,17.9,21.3,20.3,...,17.9,18.1,18.0,16.0,18.6,6.5,15.1,17.4,16.9,8.4


In [92]:
temp_max_series.to_csv(PATH_TEMP_MAX, encoding="utf8", index=False)
temp_min_series.to_csv(PATH_TEMP_MIN, encoding="utf8", index=False)

## Genera metadatos

Los metadatos deben copiarse y pegarse en un catálogo de datos abiertos, para documentar las distribuciones de series de tiempo.

In [93]:
def get_metadata_from_columns(columns, title_prefix, description_prefix, id_prefix):
    """Genera metadatos de series de tiempo a partir de las columnas."""
    
    def get_description_from_title(title):
        """Convierte nombre de una columna en descripción."""
        if title == "indice_tiempo":
            return "Indice de tiempo."
        else:
            return "{}. Estación {}.".format(
                description_prefix,
                estaciones_dict[title.split("_")[-1].upper()].title()
            )
    
    titles = list(columns)
    descriptions = [get_description_from_title(title) for title in titles]
    ids = [title.replace(title_prefix, id_prefix) for title in titles]
    
    return pd.DataFrame({
        "field_title": titles,
        "field_description": descriptions,
        "field_id": ids
    })

In [94]:
temp_min_metadata = get_metadata_from_columns(temp_min_series.columns, "temperatura_minima", "Temperatura mínima diaria", "temp_min")
temp_min_metadata.to_clipboard()
temp_min_metadata.head()

,field_title,field_description,field_id
0,indice_tiempo,Indice de tiempo.,indice_tiempo
1,temperatura_minima_saac,Temperatura mínima diaria. Estación Concordia ...,temp_min_saac
2,temperatura_minima_saag,Temperatura mínima diaria. Estación Gualeguayc...,temp_min_saag
3,temperatura_minima_saai,Temperatura mínima diaria. Estación Punta Indi...,temp_min_saai
4,temperatura_minima_saaj,Temperatura mínima diaria. Estación Junin Aero.,temp_min_saaj


In [95]:
temp_max_metadata = get_metadata_from_columns(temp_max_series.columns, "temperatura_maxima", "Temperatura máxima diaria", "temp_max")
temp_max_metadata.to_clipboard()
temp_max_metadata.head()

,field_title,field_description,field_id
0,indice_tiempo,Indice de tiempo.,indice_tiempo
1,temperatura_maxima_saac,Temperatura máxima diaria. Estación Concordia ...,temp_max_saac
2,temperatura_maxima_saag,Temperatura máxima diaria. Estación Gualeguayc...,temp_max_saag
3,temperatura_maxima_saai,Temperatura máxima diaria. Estación Punta Indi...,temp_max_saai
4,temperatura_maxima_saaj,Temperatura máxima diaria. Estación Junin Aero.,temp_max_saaj
